In [21]:
import pandas as pd
import  numpy as np
from sklearn.metrics import confusion_matrix
import category_encoders as ce
from sklearn import preprocessing
from  sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,StackingClassifier,VotingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.linear_model import LogisticRegression

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns

In [22]:
Train_data =pd.read_csv("Data/traindata.csv")
X_test=pd.read_csv("Data/testdata.csv")

In [23]:
Train_data.head(5)

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,tcp,http,SF,295,2584,0,0,0,...,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21,0
1,1,0,tcp,http,SF,312,839,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21,0
2,2,0,tcp,ftp,SF,1225,2451,0,0,0,...,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11,1
3,3,0,tcp,http,SF,293,296,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21,0
4,4,0,tcp,X11,REJ,0,0,0,0,0,...,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11,0


In [24]:
Train_data.shape

(23002, 44)

In [25]:
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23002 entries, 0 to 23001
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           23002 non-null  int64  
 1   duration                     23002 non-null  int64  
 2   protocol_type                23002 non-null  object 
 3   service                      23002 non-null  object 
 4   flag                         23002 non-null  object 
 5   src_bytes                    23002 non-null  int64  
 6   dst_bytes                    23002 non-null  int64  
 7   land                         23002 non-null  int64  
 8   wrong_fragment               23002 non-null  int64  
 9   urgent                       23002 non-null  int64  
 10  hot                          23002 non-null  int64  
 11  num_failed_logins            23002 non-null  int64  
 12  logged_in                    23002 non-null  int64  
 13  num_compromised 

In [26]:
Train_data.duplicated().sum()

0

In [27]:
Train_data["Class"].value_counts()

Class
0    12001
1    11001
Name: count, dtype: int64

# Test Data

In [28]:
X_test.shape

(22501, 43)

In [29]:
X_test.head(5)

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level
0,0,4,tcp,pop_3,SF,30,93,0,0,0,...,232,0.91,0.04,0.00,0.00,0.0,0.0,0.0,0.0,16
1,1,0,tcp,private,S0,0,0,0,0,0,...,5,0.02,0.08,0.00,0.00,1.0,1.0,0.0,0.0,21
2,2,0,udp,private,SF,52,54,0,0,0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,17
3,3,0,tcp,http,SF,367,588,0,0,0,...,255,1.00,0.00,0.12,0.06,0.0,0.0,0.0,0.0,21
4,4,0,icmp,ecr_i,SF,1008,0,0,0,0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,18


# Label Encoding

In [30]:
encoder_protocol_type = preprocessing.LabelEncoder()

Train_data['protocol_type']= encoder_protocol_type.fit_transform(Train_data['protocol_type'])
X_test['protocol_type']= encoder_protocol_type.transform(X_test['protocol_type'])

In [31]:
X_test.drop(columns=['service'],inplace=True)
Train_data.drop(columns=['service'],inplace=True)

In [32]:
encoder_flag = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
Train_data['flag']= encoder_flag.fit_transform(Train_data['flag'])
X_test['flag']= encoder_flag.transform(X_test['flag'])

# Drop ID column from train and test data [has many unique values

In [33]:
X_test.drop(columns=['ID'],inplace=True)
Train_data.drop(columns=['ID'],inplace=True)

In [34]:
y=Train_data["Class"]
x=Train_data.iloc[:, :-1]

# Apply Logistic regression binary classification

In [35]:
class_weights = {0: 1, 1: 99}


# Apply GridSearch with LogisticRegression

In [49]:
# Define the hyperparameter grid you want to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],  # List of candidate values for the regularization parameter
    'solver': ['lbfgs', 'liblinear'],  # List of candidate solvers
    'max_iter': [100, 500, 1000],  # List of candidate max_iter values
    'penalty': ['l1', 'l2'],  # Regularization penalty (L1 or L2)
    'class_weight': ['balanced', None],  # Class weight options
    'tol': [1e-4, 1e-3, 1e-2]  # Tolerance for stopping criteria
}

# Initialize the logistic regression model
base_model = LogisticRegression(random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, scoring='accuracy')

# Train the model using grid search
grid_search.fit(x, y, sample_weight=np.array([class_weights[y] for y in y]))

# Get the best hyperparameters found by grid search
best_params = grid_search.best_params_

# Get the best model found by grid search
best_model = grid_search.best_estimator_

# Print the best hyperparameters and the corresponding model
print("Best Hyperparameters:", best_params)
print("Best Model:", best_model)

C:\Users\DEBI\anaconda3\envs\My_Env_3.9\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DEBI\anaconda3\envs\My_Env_3.9\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

Best Hyperparameters: {'C': 10, 'class_weight': None, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0001}
Best Model: LogisticRegression(C=10, penalty='l1', random_state=42, solver='liblinear')


C:\Users\DEBI\anaconda3\envs\My_Env_3.9\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
450 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\DEBI\anaconda3\envs\My_Env_3.9\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\DEBI\anaconda3\envs\My_Env_3.9\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\DEBI\anaconda3\envs\My_Env_3.9\lib\site-packages\sklearn\linear_model\_log

# best params from grid_search

In [53]:
best_params

{'C': 10,
 'class_weight': None,
 'max_iter': 100,
 'penalty': 'l1',
 'solver': 'liblinear',
 'tol': 0.0001}

# run with best model

In [56]:

y_pred = best_model.predict(X_test)

In [57]:
OUTPUT_sample=pd.DataFrame({"Class":y_pred})
OUTPUT_sample=OUTPUT_sample.rename_axis("ID")
OUTPUT_sample
OUTPUT_sample.to_csv("sampleSubmission_Model_1.csv")